In [ ]:
# 내 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Colab에서 GitHub 에 있는 데이터 가져오기
# caffemodel 다운로드 URL : https://github.com/habom2310/People-tracking-with-Age-and-Gender-detection
# Clone Web URL 를 복사해 옵니다.

!git clone https://github.com/habom2310/People-tracking-with-Age-and-Gender-detection.git

Cloning into 'People-tracking-with-Age-and-Gender-detection'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 74 (delta 6), reused 15 (delta 3), pack-reused 50
Unpacking objects: 100% (74/74), done.


In [ ]:
# 파일 복사
!cp People-tracking-with-Age-and-Gender-detection/age_gender_models/age_net.caffemodel gdrive/My\ Drive/CV/Age\ and\ Gender\ Recognition/
!cp People-tracking-with-Age-and-Gender-detection/age_gender_models/deploy_age.prototxt gdrive/My\ Drive/CV/Age\ and\ Gender\ Recognition/
!cp People-tracking-with-Age-and-Gender-detection/age_gender_models/gender_net.caffemodel gdrive/My\ Drive/CV/Age\ and\ Gender\ Recognition/
!cp People-tracking-with-Age-and-Gender-detection/age_gender_models/deploy_gender.prototxt gdrive/My\ Drive/CV/Age\ and\ Gender\ Recognition/
print('4 files copy complete!!')

4 files copy complete!!


In [ ]:
# 다운로드 받았던 파일 제거(선택사항)
!rm -r People-tracking-with-Age-and-Gender-detection

In [ ]:
# 필요한 패키지와 모듈을 불러옴
import glob
import dlib
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']  # 나이 분류 구간 정의
gender_list = ['Male','Female']  # 성별 구분 정의

In [ ]:
detector = dlib.get_frontal_face_detector()       # 얼굴을 감지하는 detector 정의

age_detector = cv2.dnn.readNetFromCaffe(          # 나이를 감지하는 detector 정의
               'gdrive/My Drive/CV/Age and Gender Recognition/deploy_age.prototxt', 
               'gdrive/My Drive/CV/Age and Gender Recognition/age_net.caffemodel')
gender_detector = cv2.dnn.readNetFromCaffe(       # 성별을 감지하는 detector 정의
               'gdrive/My Drive/CV/Age and Gender Recognition/deploy_gender.prototxt', 
               'gdrive/My Drive/CV/Age and Gender Recognition/gender_net.caffemodel')

img_list = glob.glob('gdrive/My Drive/CV/Age and Gender Recognition/images/*.jpg')  # 이미지 디렉토리의 리스트들을 불러온다.

In [ ]:
# 이미지 폴더의 파일 수 확인
print('image folder files count : %s' % len(img_list))

image folder files count : 15


In [ ]:
for img_path in img_list:
  image = cv2.imread(img_path)  # 이미지를 읽어 옵니다.
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  faces = detector(gray, 1)  # detector 에 의해 얼굴 감지
  print("Number of faces detected: {}".format(len(faces)))

  for face in faces:
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()  # boxing 좌표를 구한다.

    face_img = image[y1:y2, x1:x2].copy()  # 이미지에서 얼굴 영역만 copy

    blob = cv2.dnn.blobFromImage(face_img, scalefactor=1, size=(227, 227),
           mean=(78.4263377603, 87.7689143744, 114.895847746), swapRB=False, crop=False)

    # predict age(나이를 예측합니다)
    age_detector.setInput(blob)
    age_preds = age_detector.forward()
    age = age_list[age_preds[0].argmax()]

    # predict gender(성별을 예측합니다)
    gender_detector.setInput(blob)
    gender_preds = gender_detector.forward()
    gender = gender_list[gender_preds[0].argmax()]

    cv2.rectangle(image, (x1, y1), (x2, y2), (255,255,255), 2)
    overlay_text = '%s %s' % (gender, age)
    cv2.putText(image, overlay_text, org=(x1, y1), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0,0,0), thickness=10)
    cv2.putText(image, overlay_text, org=(x1, y1),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255), thickness=2)

  # 나이와 성별이 추정된 이미지 Display  
  cv2_imshow(image)
  print("\n\n")

Output hidden; open in https://colab.research.google.com to view.